
# A Notebook to prepare gridded climate time series data for DHSVM modeling of the Skagit Watershed  <br />
This workflow was originally developed to digitally observe the Sauk-Suiattle Watershed, and is expanded here to the Skagit watershed. Low elevation observations are used to correct modeled atmospheric data based on the differences in the long term mean monthly temperature and precipitation. This is expected to improve hydrologic modeling at low elevations, and we are prioritizing predictions of low flows in low elevation tributaries. Limited analysis conducted to date shows that this approach may not be an accurate representation of high elevation processes.  This is an area of active research.
<br /><br />
<img src= "http://www.sauk-suiattle.com/images/Elliott.jpg"
style="float:left;width:200px;padding:20px">   

*Use this Jupyter Notebook to:* <br /> 
Download and generate lists of gridded climate points<br />
Download Livneh daily 1/16 degree gridded climate data, <br /> 
Download WRF daily 1/16 degree gridded climate data, <br /> 
Visualize daily, monthly, and annual temperature and precipitation data. <br /> 
Calculate Long-term Mean Monthly Bias Corrections for WRF using Livneh Low Elevation data<br /> 
Bias correct each Livneh grid cell using bias corrected WRF (use to correct Livneh 2013 and MACA data). <br />
Visualize daily, monthly, and annual temperature and precipitation data with corrected results. <br /> 
Update VIC model soil input (optional). <br /> 
Save results back to HydroShare. <br /> 
 <br /> <br /> <img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:right;width:120px;padding:20px">  
#### A Watershed Dynamics Model by the Watershed Dynamics Research Group in the Civil and Environmental Engineering Department at the University of Washington 

## 1.  HydroShare Setup and Preparation

To run this notebook, we must import several libaries. These are listed in order of 1) Python standard libraries, 2) hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading and creation, and 3) the observatory_gridded_hydromet library that is downloaded with this notebook. 

In [ ]:
#Python libraries available on CUAHSI JupyterHub 
import os
import numpy as np
import pandas as pd
import json
from datetime import datetime, timedelta

%matplotlib inline
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

#HydroShare Utilities
from utilities import hydroshare

In [ ]:
start_time=datetime.now()

### UPDATE ME 1b.  Import python scripts from a Github repository
#### Please see the [Observatory]( https://github.com/Freshwater-Initiative/Observatory/blob/master/README.md) repository on Github with a Readme instructions on how to use Git and the JupyterHub server terminal to push/pull changes.   After completing the steps to get the observatory_gridded_hydromet.py script into your HydroShare Utilities folder, execute the next cell. 

In [ ]:
import ogh

Establish a secure connection with HydroShare by instantiating the hydroshare class that is defined within hs_utils. In addition to connecting with HydroShare, this command also sets and prints environment variables for several parameters that will be useful for saving work back to HydroShare. 

In [ ]:
hs=hydroshare.hydroshare()
homedir = ogh.mapContentFolder(str(os.environ["HS_RES_ID"]))
print('Data will be loaded from and save to:'+homedir)

If you are curious about where the data is being downloaded, click on the Jupyter Notebook dashboard icon to return to the File System view.  The homedir directory location printed above is where you can find the data and contents you will download to a HydroShare JupyterHub server.  At the end of this work session, you can migrate this data to the HydroShare iRods server as a Generic Resource. 

## 2. Download and generate lists of gridded climate points for a watershed
Retrieve a list of grid points and configuration file from a HydroShare resource
This example uses a ascii text that is stored in HydroShare at the following url: https://www.hydroshare.org/resource/d90289409f904017831d308642c1eb30/ . The data for our processing routines can be retrieved using the getResourceFromHydroShare function by passing in the global identifier from the url above.  In the next cell, we download this resource from HydroShare, and identify that the table in this resource is the 'mappingfile' variable identifying the Lat/Long points to be used for downloading hydrometeorology data.  The file must include columns with station numbers (this can be aribitrary), latitude, longitude, and elevation. The header of these columns must be FID, LAT, LONG_, and ELEV or RASTERVALU, respectively. The station numbers will be used for the remainder of the code to uniquely reference data from each climate station, as well as to identify minimum, maximum, and average elevation of all of the climate stations.

### User provides their HydroShare resource ID from their own polgyon shapefile uploaded to HydroShare

In [ ]:
hs.getResourceFromHydroShare('b4c129d29e5e4452985c9d8dc0fe01ed')
shapefile = hs.content['SkagitRiver_BasinBoundary.shp']

### Import the exising point shapefile of available 1/16 degree grid centroid locations shared on HydroShare as a public resource

In [ ]:
hs.getResourceFromHydroShare('ef2d82bf960144b4bfb1bae6242bcc7f')
NAmer = hs.content['NAmer_dem_list.shp']

### Use TreatGeoSelf to generate a list of lat/long points in your area of interest
The TreatGeoSelf() function was designed to easily generate a list of lat/long points in your area of interest
Use the buffer distance in decimal degrees to select points within your polygon (distance=0) or within a 1/16 degree buffer outside of the polygon (distance = 0.0625).

In [ ]:
mappingfile = ogh.treatgeoself(shapefile=shapefile, NAmer=NAmer, folder_path=os.getcwd(), outfilename='monkeysonatree.csv', buffer_distance=0.00)
print(mappingfile)

### Provide Location Name and watershed drainage area (m2)
#### Watershed information upstream of USGS 12200500 Skagit River near Mount Vernon, WA  
Location.--Latitude 48°26'42", Longitude 122°20'03", in SE 1/4 SE 1/4 Section 7, Township 34 North, Range 4 East, in Skagit County, Hydrologic Unit 17110007, on right bank 220 feet downstream of bridge on U.S. Highway 99, 1.5 miles north of Skagit Valley Junior College in Mount Vernon, and at river mile 15.7. Drainage area is 3,093 mi2, of which 400 mi2 is in Canada. Datum of gage is NGVD of 1929. 
https://waterdata.usgs.gov/nwis/uv?site_no=12200500

In [ ]:
loc_name='Skagit Watershed'
streamflow_watershed_drainage_area=8010833000 # square meters

# Read in the Observatory metadata file 
This file contains the variables, data types and metadata related to Livneh et al., 2013; 2015 and Salathe et al., 2014 gridded hydrometeorology products. 

In [ ]:
#Assuming this is pulled from Github, how can we import this from Utilities.
#Otherwise it needs to be in each HydroShare resource - which if fine too. 
with open('ogh_meta.json','r') as r:
    meta_file = json.load(r)
    r.close()

sorted(meta_file.keys())

 ## 3. Download climate data 

### Get Daily Meteorologic Data (1915-2011) from Livneh et al. 2013 

The functions used in this section apply to hydrometeorology data within the Continental United States with daily data 1915-2011. <br/>
View data extent at  Livneh, B. (2017). Gridded climatology locations (1/16th degree): Continental United States extent, HydroShare, http://www.hydroshare.org/resource/14f0a6619c6b45cc90d1f8cabc4129af

Please cite: <br/>
Livneh B., E.A. Rosenberg, C. Lin, B. Nijssen, V. Mishra, K.M. Andreadis, E.P. Maurer, and D.P. Lettenmaier, 2013: A Long-Term Hydrologically Based Dataset of Land Surface Fluxes and States for the Conterminous United States: Update and Extensions, Journal of Climate, 26, 9384–9392.<br/>
<br/>
The getClimateData_DailyMET_livneh2013() function reads in the mapping file table, downloads, and unzips the data files for each of the longitude and latitude points. The folder containing the data is within the directory listed as homedir. 

In [ ]:
help(ogh.getDailyMET_livneh2013)

In [ ]:
Daily_MET_1915_2011 = ogh.getDailyMET_livneh2013(homedir, mappingfile)


### Get Daily Weather Research and Forecasting (WRF 1950-2010 Pacific Northwest) from Salathe et al., 2014
<br/>
Please cite 2014 data using: <br/>
Salathé, EP, AF Hamlet, CF Mass, M Stumbaugh, S-Y Lee, R Steed: 2017. Estimates of 21st Century Flood Risk in the Pacific Northwest Based on Regional Scale Climate Model Simulations.  J. Hydrometeorology. DOI: 10.1175/JHM-D-13-0137.1

This data is also available on HydroShare and can be downloaded using the following line of code (copy into code block):
hs.getResourceFromHydroShare('0db969e4cfb54cb18b4e1a2014a26c82')


In [ ]:
help(ogh.getDailyWRF_salathe2014)

In [ ]:
Daily_WRFraw_1950_2010 = ogh.getDailyWRF_salathe2014(homedir, mappingfile)

Helpful hint: Jupyter Notebooks on the CUAHSI JupyterHub server on ROGER supercomputer can use Python or bash command line coding to explore the data folders.  Alternatively, you can click on the orange Jupyter icon in the upper left corner to open the folder view to see where in the world your files are.

In [ ]:
print('This is the list of folders in your directory for this HydroShare resource.')
test = [each for each in os.listdir(homedir) if os.path.isdir(each)]
print(test)

## 4.  Calculate Long-term Monthly Bias Corrections for WRF using Livneh Low Elevation data

In [ ]:
# take two tuples representing start and end date ranges, then find their overlapping date_range
dr1 = meta_file['dailymet_livneh2013']['date_range']
dr2 = meta_file['dailywrf_salathe2014']['date_range']

dr = ogh.overlappingDates(tuple([dr1['start'],dr1['end']]), tuple([dr2['start'],dr2['end']]))
dr

In [ ]:
#initiate new dictionary with original data
ltm_0to3000 = ogh.gridclim_dict(gridclim_folder='livneh2013_MET',
                               loc_name=loc_name,
                               dataset='dailymet_livneh2013',
                               mappingfile=mappingfile, 
                               metadata=meta_file,
                               file_start_date=None, 
                               file_end_date=None,
                               subset_start_date=dr[0],
                               subset_end_date=dr[1])

ltm_0to3000 = ogh.gridclim_dict(gridclim_folder='livneh2013_MET',
                               loc_name=loc_name,
                               dataset='dailywrf_salathe2014',
                               mappingfile=mappingfile, 
                               metadata=meta_file,
                               file_start_date=None, 
                               file_end_date=None,
                               subset_start_date=dr[0],
                               subset_end_date=dr[1],
                               df_dict=ltm_0to3000)

sorted(ltm_0to3000.keys())

## 5. Perform bias correction using differences between WRFbc (from 4) and Liv2013

In [ ]:
BiasCorr_wrfbc = ogh.compute_diffs(df_dict=ltm_0to3000, df_str='0to3000m',
                                   gridclimname1='dailywrf_salathe2014',
                                   gridclimname2='dailymet_livneh2013',
                                   prefix2=['month'],
                                   prefix1=meta_file['dailymet_livneh2013']['variable_list'])

BiasCorr_wrfbc_P = ogh.compute_ratios(df_dict=ltm_0to3000, df_str='0to3000m',
                                      gridclimname1='dailywrf_salathe2014',
                                      gridclimname2='dailymet_livneh2013',
                                      prefix2=['month'],
                                      prefix1=meta_file['dailymet_livneh2013']['variable_list'])

           
BiasCorr_wrfbc['PRECIP_0to3000m'] = BiasCorr_wrfbc_P['PRECIP_0to3000m']

print('Precipitation values are a ratio of WRF_m/Liv_m and Temperature values are the difference between WRF_m-Liv_m')

print(sorted(BiasCorr_wrfbc.keys()))


### Export the BiasCorr_wrfbc dictionary of dataframes

In [ ]:
ogh.saveDictOfDf(outfilename='BiasCorr_wrfbc.json', dictionaryObject=BiasCorr_wrfbc)

In [ ]:
### to read biascorr json objects back in, use readDictOfDF
#testingobject = readDictOfDf(infilename='BiasCorr_wrfbc_test.json')
#testingobject

In [ ]:
Daily_MET_1915_2011_WRFbc_liv, meta_file = ogh.makebelieve(homedir=homedir,
                                                           mappingfile=mappingfile,
                                                           BiasCorr=BiasCorr_wrfbc,
                                                           metadata=meta_file,
                                                           start_catalog_label='dailymet_livneh2013',
                                                           end_catalog_label='dailymet_livneh2013_wrfbc', 
                                                           file_start_date=None,
                                                           file_end_date=None,
                                                           data_dir=None,
                                                           dest_dir_suffix='biascorrWRF_liv')

In [ ]:
#initiate new dictionary with original data
ltm_0to3000 = ogh.gridclim_dict(gridclim_folder='livneh2013_MET',
                                loc_name=loc_name,
                                dataset='dailymet_livneh2013_wrfbc',
                                mappingfile=mappingfile, 
                                metadata=meta_file,
                                file_colnames=None,
                                file_delimiter=None,
                                file_start_date=None, 
                                file_end_date=None,
                                file_time_step=None,
                                subset_start_date=dr[0],
                                subset_end_date=dr[1],
                                df_dict=ltm_0to3000)

#sorted(ltm_0to3000.keys())

## 6. Low elevation bias correction

Use monthly averages to correct the WRF values by the low elevation Livneh data.   Below, two new bias correction dataframes the same shape as BiasCorr are created to test the difference between correcting to Tmin and Tmax (global), and correcting both Tmin and Tmax by Tavg (global_Tavg).

In [ ]:
print(BiasCorr_wrfbc['PRECIP_0to3000m'].shape)
#Make two new bias correction dataframes the same shape as BiasCorr
BiasCorr_wrfbc_lowLiv=BiasCorr_wrfbc

#Files used to generate low elevation correction factors
#hs.getResourceFromHydroShare('ff886a1e191e47fd9ba13c23922741da')
#shapefile_low = hs.content['SkagitLowElevationGrid_lessthan550_15k_poly.shp']
#The monthly mean values are calculated in Observatory_Sauk_LivBC2WRFlow_092317.ipynb for 75 climate grid cells within a 15km buffer of the Skagit Watershed and are less than 550m
global_lowelev_precip=[-2.840475,-2.513567,-2.397790,-2.276174,-1.639153,-1.589924,-0.686511,-0.541030,-0.750094,-1.777611,-2.442535,-2.629715]
global_lowelev_tmax=[1.799996,2.804549,3.695474,3.849152,3.673436,3.562770,3.835815,3.839669,2.552046,1.718202,1.442429,1.314274]
global_lowelev_tmin=[-1.298700,-1.045357,-0.143184,-0.127852,0.371726,0.868289,0.427259,0.085887,-1.208515,-2.180547,-1.978898,-1.578709]
global_lowelev_tavg=[0.250648,0.879596,1.776145,1.860650,2.022581,2.215530,2.131537,1.962778,0.671766,-0.231173,-0.268234,-0.132217]
global_wind=[0,0,0,0,0,0,0,0,0,0,0,0]

for column in BiasCorr_wrfbc_lowLiv['PRECIP_0to3000m']:
    BiasCorr_wrfbc_lowLiv['PRECIP_0to3000m'].ix[1:12,column]=global_lowelev_precip
    BiasCorr_wrfbc_lowLiv['TMAX_0to3000m'].ix[1:12,column]=global_lowelev_tmax
    BiasCorr_wrfbc_lowLiv['TMIN_0to3000m'].ix[1:12,column]=global_lowelev_tmin
    BiasCorr_wrfbc_lowLiv['WINDSPD_0to3000m'].ix[1:12,column]=global_wind
print(BiasCorr_wrfbc_lowLiv['PRECIP_0to3000m'])
print(BiasCorr_wrfbc_lowLiv['TMAX_0to3000m'])
print(BiasCorr_wrfbc_lowLiv['TMIN_0to3000m'])
print(BiasCorr_wrfbc_lowLiv['WINDSPD_0to3000m'])  

Daily_MET_1915_2011_WRFbc_liv_global, meta_file = ogh.makebelieve(homedir=homedir,
                                                                  mappingfile=mappingfile,
                                                                  BiasCorr=BiasCorr_wrfbc_lowLiv,
                                                                  metadata=meta_file,
                                                                  start_catalog_label='dailymet_livneh2013_wrfbc',
                                                                  end_catalog_label='dailymet_livneh2013_wrfbc_global', 
                                                                  file_start_date=None,
                                                                  file_end_date=None,
                                                                  data_dir=None,
                                                                  dest_dir_suffix='biascorrWRF_global')



### Save the correction factors for use with future climate data corrections using MACA Livneh

In [ ]:
ogh.saveDictOfDf(outfilename='BiasCorr_wrfbc_lowLiv.json', dictionaryObject=BiasCorr_wrfbc_lowLiv)

### Add the corrected time series to the dictionary with the raw datasets

In [ ]:
ltm_0to3000 = ogh.gridclim_dict(gridclim_folder='livneh2013_MET',
                                loc_name=loc_name,
                                dataset='dailymet_livneh2013_wrfbc_global',
                                mappingfile=mappingfile, 
                                metadata=meta_file,
                                file_colnames=None,
                                file_delimiter=None,
                                file_start_date=None, 
                                file_end_date=None,
                                file_time_step=None,
                                subset_start_date=dr[0],
                                subset_end_date=dr[1],
                                df_dict=ltm_0to3000)

#### List of variables and statistics calculated; available for visualization and analysis

In [ ]:
sorted(ltm_0to3000.keys())

In [ ]:
temp = pd.read_csv(mappingfile).sort_values('ELEV')
temp.head(10)

### Plot and compare monthly maximum temperature

In [ ]:
# let's compare monthly averages for TMAX using livneh, salathe, and the salathe-corrected livneh
comp = ['month_TMAX_dailymet_livneh2013','month_TMAX_dailymet_livneh2013_wrfbc','month_TMAX_dailywrf_salathe2014','month_TMAX_dailymet_livneh2013_wrfbc_global']

obj = dict()
for eachkey in ltm_0to3000.keys():
    if eachkey in comp:
        obj[eachkey] = ltm_0to3000[eachkey] 
panel_obj = pd.Panel.from_dict(obj)
print(panel_obj)

fig, ax = plt.subplots()
lws=[3, 10, 3, 3]
styles=['b--','go-','y--','ro-']

for col, style, lw in zip(comp, styles, lws):
    #panel_obj.xs(key=(0.0, 48.53125, -121.59375), axis=2)[col].plot(style=style, lw=lw, ax=ax, legend=True)
    #0    0  48.84375 -121.15625  1483.0

    #0, 49.28125, -120.84375, 1727.0
    panel_obj.xs(key=(0, 49.28125, -120.84375), axis=2)[col].plot(style=style, lw=lw, ax=ax, legend=True)    

fig, ax = plt.subplots()
lws=[3, 10, 3, 3]
styles=['b--','go-','y--','ro-']

for col, style, lw in zip(comp, styles, lws):
    #    74	48.59375	-121.71875	305
    
    # 176, 48.46875, -122.28125, 16.0
    panel_obj.xs(key=(176, 48.46875, -122.28125), axis=2)[col].plot(style=style, lw=lw, ax=ax, legend=True)    
    

### Plot and compare monthly precipitation

In [ ]:
# let's compare monthly averages for TMAX using livneh, salathe, and the salathe-corrected livneh
comp = ['month_PRECIP_dailymet_livneh2013','month_PRECIP_dailymet_livneh2013_wrfbc','month_PRECIP_dailywrf_salathe2014','month_PRECIP_dailymet_livneh2013_wrfbc_global']

obj = dict()
for eachkey in ltm_0to3000.keys():
    if eachkey in comp:
        obj[eachkey] = ltm_0to3000[eachkey] 
panel_obj = pd.Panel.from_dict(obj)
print(panel_obj)

fig, ax = plt.subplots()
lws=[3, 10, 3, 3]
styles=['b--','go-','y--','ro-']

for col, style, lw in zip(comp, styles, lws):
    #panel_obj.xs(key=(0.0, 48.53125, -121.59375), axis=2)[col].plot(style=style, lw=lw, ax=ax, legend=True)
    
    #0.0, 48.84375, -121.15625
    panel_obj.xs(key=(0, 49.28125, -120.84375), axis=2)[col].plot(style=style, lw=lw, ax=ax, legend=True)    
    
    
fig, ax = plt.subplots()
lws=[3, 10, 3, 3]
styles=['b--','go-','y--','ro-']

for col, style, lw in zip(comp, styles, lws):
    
    #74.0, 48.59375, -121.71875
    panel_obj.xs(key=(176, 48.46875, -122.28125), axis=2)[col].plot(style=style, lw=lw, ax=ax, legend=True)    
  

## 6. (optional) - Run MetSim to dissagregate daily data to 3-hrly DHSVM inputs

See [Observatory_Sauk_MetSim_Python3.ipynb](https://jupyter.cuahsi.org/user/christinabandaragoda/notebooks/notebooks/data/f0f90f5645864e0d9c0e0209d0095d74/f0f90f5645864e0d9c0e0209d0095d74/data/contents/Observatory_Sauk_MetSim_Python3.ipynb) 
Edit this Markdown code hyperlink to your User Name if the Sauk Observatory HydroShare resource has been downloaded to your CUAHSI JupyterHub server personal user space.



## 7. (optional) - Update VIC model input file: soil

In [ ]:
ogh.switchUpVICSoil(input_file=os.path.join(homedir,'soil_base'),
                    output_file='soil',
                    mappingfile=mappingfile,
                    homedir=homedir)

## 8. Save the results back into HydroShare
<a name="creation"></a>

Using the `hs_utils` library, the results of the Geoprocessing steps above can be saved back into HydroShare.  First, define all of the required metadata for resource creation, i.e. *title*, *abstract*, *keywords*, *content files*.  In addition, we must define the type of resource that will be created, in this case *genericresource*.  

***Note:*** Make sure you save the notebook at this point, so that all notebook changes will be saved into the new HydroShare resource.

In [ ]:
print('This is the list of folders in your directory for this HydroShare resource.')
test = [each for each in os.listdir(homedir) if os.path.isdir(each)]
print(test)

Move each file on the server within the 'files' list to an :EXISTING" HydroShare Generic Resource content folder.  Parent_resource is the destination resource ID for an existing Generic Resource. Files is a list of filepaths.

In [ ]:
ThisNotebook='Observatory_Skagit_LivBC2WRFlow_122117.ipynb' #check name for consistency

liv2013_tar = 'livneh2013.tar.gz'
wrf_tar = 'salathe2014.tar.gz'
biascorrWRF_liv_tar = 'biascorrWRF_liv.tar.gz'
biascorrWRF_global_tar = 'biascorrWRF_global.tar.gz'

!tar -zcf {liv2013_tar} livneh2013
!tar -zcf {wrf_tar} salathe2014
!tar -zcf {biascorrWRF_liv_tar} biascorrWRF_liv
!tar -zcf {biascorrWRF_global_tar} biascorrWRF_global

observatory_gridded_hydromet='ogh.py'
soil = 'soil'
CorrectionFactors_wrfliv='BiasCorr_wrfbc.json'
CorrectionFactors_lowliv='BiasCorr_wrfbc_lowLiv.json'
listofgridpoints ='monkeysonatree.csv'

files=[ThisNotebook,
       liv2013_tar,
       observatory_gridded_hydromet,
       wrf_tar,
       biascorrWRF_liv_tar,biascorrWRF_global_tar,
       soil,listofgridpoints,
       CorrectionFactors_wrfliv,CorrectionFactors_lowliv]

In [ ]:
# for each file downloaded onto the server folder, move to a new HydroShare Generic Resource
title = 'Skagit Observatory Bias Correction Results - Livneh et al., 2013 to WRF (Salathe et al., 2014) and low elevation spatial average correction.'
abstract = 'This output is a bias correction test to generate a hybrid gridded meteorology product. This dataset was generated December 21, 2017 using Observatory code from https://github.com/ChristinaB/Observatory.'
keywords = ['Sauk', 'climate', 'WRF','hydrometeorology'] 
rtype = 'genericresource'  

# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title,
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=False)